In [3]:
import sys
import logging
import pymysql as sq
import boto3
import psycopg2
import pandas as pd
import json
import http

In [2]:
ENDPOINT = "sandbox-db-cluster.cluster-cf6pgrepnkpy.us-east-1.rds.amazonaws.com"
PORT = 5432
USR = "postgres"
REGION = "us-east-1"
DBNAME = "BungeeMatchPool"
PASS= "Bungee_sand_rds_2021!"
db_table_name = "brand_standardization"


In [150]:

def connect_to_database():
    try:
        ENDPOINT = "sandbox-db-cluster.cluster-cf6pgrepnkpy.us-east-1.rds.amazonaws.com"
        PORT = 5432
        USR = "postgres"
        DBNAME = "BungeeMatchPool"
        PASS= "Bungee_sand_rds_2021!"
        conn = psycopg2.connect(host=ENDPOINT, port=PORT, database=DBNAME, user=USR, password=PASS)
        
        return conn
        # cur.execute("""SELECT * from brand_standardization""")
        # query_results = cur.fetchall()
        # print(f'Query results: {query_results}')
        # print(type(query_results))
    
    except Exception as error:
        print(f'in Creating connection ! {error}')

In [13]:
def open_file(s3_bucket, s3_key):    
    print("Importing file from s3")
    path = 's3://{}/{}'.format(s3_bucket, s3_key)
    print('IMporting file from {}'.format(path))
    df = pd.read_parquet(path=s3_key)
    df = df[["cluster_id", "crawled_name", "source", "status", "updated_date", "product_url_1", "image_url_1", "product_url_2"]]
    df = df.fillna("")
    df.insert(2, "standard_name", "")
    df.rename(columns = {'image_url_1':'image_url'}, inplace = True)
    print("File loaded sucessfully :", len(df.index))
    return df
   

In [148]:
def show_db( table_name):
    db_connection = connect_to_database()
    cur = db_connection.cursor()
    cur.execute("""SELECT * from brand_standardization""")
    query_results = cur.fetchall()
    print(f'Query results: {query_results}')
    print(len(query_results))
    db_connection.close()

In [149]:
def execute_db_query( query):
    db_connection = connect_to_database()
    cur = db_connection.cursor()
    print('executing query', query)
    try : 
        cur.execute(query)
        db_connection.commit()
        print('executing finished successfully')
        db_connection.close()
    except Exception as e:
        print(e)
        db_connection.close ()
    

In [15]:

updated_brand_Cluster = open_file('', '/home/preacher/Bungee/bnt/sample_update_brands.parquet')

Importing file from s3
IMporting file from s3:////home/preacher/Bungee/bnt/sample_update_brands.parquet
File loaded sucessfully : 10


In [265]:
def get_db_query(source_df,  target_db_table, action):

    string_dtypes = source_df.convert_dtypes().select_dtypes("string")
    source_df[string_dtypes.columns] = string_dtypes.apply(lambda x: x.str.replace("'","''")).apply(lambda x: x.str.strip('"\' '))
#     source_df[string_dtypes.columns] = string_dtypes.
    if action == 'INSERT':
        print("Generating insert query")
        query  = 'INSERT INTO '+target_db_table+' ('+ str(', '.join(source_df.columns))+ ') VALUES '
        for index, row in source_df.iterrows():
            query = query + str(tuple(row.values)) + ','
        query = query[:-1]

    elif action == 'UPDATE':
        print("Generating update query")
        source_df.insert(2, "user_id", "")
        source_df.insert(2, "current_step", "")
        query = "UPDATE "+ target_db_table+ " SET "
        for column_name in source_df.columns:
            if column_name in ['crawled_name']:
                continue
            if column_name == 'updated_date':
                sub_query =  " " + column_name + " = CASE crawled_name "
                for index, row in source_df.iterrows():
                    sub_query = sub_query + "  WHEN '" + str(row.crawled_name) + "' THEN TO_TIMESTAMP('" + str(row[column_name]) + "', 'YYYY-MM-DD HH24:MI:SS.MS') "

                sub_query = sub_query + " END,"
                query = query + sub_query
            else:
                sub_query =  " " + column_name + " = CASE crawled_name "
                for index, row in source_df.iterrows():
                    sub_query = sub_query + "  WHEN '" + str(row.crawled_name) + "' THEN '" + str(row[column_name]) + "' "

                sub_query = sub_query + " END,"
                query = query + sub_query


        query = query.rstrip(query[-1])

        id_list = str("', '".join(source_df.crawled_name))
        query  = query + " WHERE crawled_name in ('" + id_list + "') "


    query = query.replace("\\","")
    print('Query generated')
    return query 


In [47]:
table_name = 'brand_standardization'

In [266]:
df = updated_brand_Cluster.head(2)

In [267]:
df.at[0,'crawled_name']= "Purnell's \"Old Fol\""

In [262]:
df

,cluster_id,crawled_name,standard_name,source,status,updated_date,product_url_1,image_url,product_url_2
0,-1,"Purnell's ""Old Fol""",,UC,IN_PROGRESS,2023-01-09 13:19:13.694,,,
1,-1,"""TBA""",,UC,IN_PROGRESS,2023-01-09 13:19:13.694,,,


In [226]:
pd.options.mode.chained_assignment = None

In [268]:
query = get_db_query(df, table_name, 'INSERT')
# execute_db_query(query)
# show_db( table_name)

Generating insert query
Query generated


In [269]:
print(query)

INSERT INTO brand_standardization (cluster_id, crawled_name, standard_name, source, status, updated_date, product_url_1, image_url, product_url_2) VALUES (-1, 'Purnell''s "Old Fol', '', 'UC', 'IN_PROGRESS', '2023-01-09 13:19:13.694', '', '', ''),(-1, 'TBA', '', 'UC', 'IN_PROGRESS', '2023-01-09 13:19:13.694', '', '', '')


In [235]:
query = query.replace("\\","")

In [236]:
print(query)

INSERT INTO brand_standardization (cluster_id, crawled_name, standard_name, source, status, updated_date, product_url_1, image_url, product_url_2) VALUES (-1, 'Purnell''s "Old Fol"', '', 'UC', 'IN_PROGRESS', '2023-01-09 13:19:13.694', '', '', ''),(-1, '"TBA"', '', 'UC', 'IN_PROGRESS', '2023-01-09 13:19:13.694', '', '', '')


In [206]:
s = 'Kaushik\'s'

In [207]:
print(s)

Kaushik's


In [208]:
print(s.replace("'","''"))

Kaushik''s


In [81]:
query = query.replace('"0""isAjaxComplete_B001IOHA8M"', 'new_brand_1')

In [82]:
query = query.replace('"TBA"', 'new_brand_2')

In [ ]:
UPDATE brand_standardization SET  cluster_id = CASE crawled_name   WHEN 'purinaagold' THEN '-1'   WHEN 'purinaone' THEN '-1'  END,
 current_step = CASE crawled_name   WHEN 'purinaagold' THEN ''   WHEN 'purinaone' THEN ''  END,
 user_id = CASE crawled_name   WHEN 'purinaagold' THEN ''   WHEN 'purinaone' THEN ''  END,
 standard_name = CASE crawled_name   WHEN 'purinaagold' THEN ''   WHEN 'purinaone' THEN ''  END,
 source = CASE crawled_name   WHEN 'purinaagold' THEN 'UC'   WHEN 'purinaone' THEN 'UC'  END,
 status = CASE crawled_name   WHEN 'purinaagold' THEN 'IN_PROGRESS'   WHEN 'purinaone' THEN 'IN_PROGRESS'  END,
 updated_date = CASE crawled_name   WHEN 'purinaagold' THEN '2023-01-09 13:19:13.694'   WHEN 'purinaone' THEN '2023-01-09 13:19:13.694'  END,
 product_url_1 = CASE crawled_name   WHEN 'purinaagold' THEN 'new_product_url_1'   WHEN 'purinaone' THEN 'new_product_url_1'  END,
 image_url = CASE crawled_name   WHEN 'purinaagold' THEN 'new_image_url'   WHEN 'purinaone' THEN 'new_image_url'  END,
 product_url_2 = CASE crawled_name   WHEN 'purinaagold' THEN 'new_product_url_2'   WHEN 'purinaone' THEN 'new_product_url_2'  END, WHERE crawled_name in ('purinaagold', 'purinaone') 

In [81]:
new_brand_cluster = open_file('', '/home/preacher/Bungee/bnt/data_set/database/updation.csv')

In [82]:
new_brand_cluster.columns

Index(['id', 'crawled_name', 'standard_name', 'cluster_id', 'status',
       'user_id', 'updated_date', 'source', 'product_url_1', 'product_url_2',
       'image_url', 'current_step'],
      dtype='object')

In [113]:
target_db_table = 'target_db_table'
query = 'UPDATE '+ target_db_table+ ' SET \n'
for column_name in new_brand_cluster.columns:
    if column_name in ['id', 'user_id', 'product_url_1', 'product_url_2', 'image_url', 'current_step', 'standard_name']:
        continue
    sub_query =  '\t' + column_name + ' = CASE id \n'
    for index, row in new_brand_cluster.iterrows():
        sub_query = sub_query + "\t\t WHEN '" + str(row.id) + "' THEN '" + str(row[column_name]) + "' \n"
    sub_query = sub_query + '\tEND \n'

    query = query + sub_query
id_list = str(', '.join(new_brand_cluster.id))
query  = query + ' WHERE id in (' + id_list + ') '
print(query)
    

UPDATE target_db_table SET 
	crawled_name = CASE id 
		 WHEN '6e49474c-8103-11ed-8410-ebb084c2a71f' THEN 'NUTRO MAX' 
		 WHEN '742b0f74-8103-11ed-8410-ebb084c2a71f' THEN 'NUTRO Ultra' 
		 WHEN 'a67d5688-5ee1-11ed-9f7e-5fce40501278' THEN 'purrrr' 
		 WHEN 'd2a19802-4fbc-11ed-9c57-a181204819a4' THEN 'Purina pro plan' 
		 WHEN '890d0180-5c01-11ed-b331-2b7bf508673' THEN 'Abc' 
	END 
	cluster_id = CASE id 
		 WHEN '6e49474c-8103-11ed-8410-ebb084c2a71f' THEN '45' 
		 WHEN '742b0f74-8103-11ed-8410-ebb084c2a71f' THEN '45' 
		 WHEN 'a67d5688-5ee1-11ed-9f7e-5fce40501278' THEN '45' 
		 WHEN 'd2a19802-4fbc-11ed-9c57-a181204819a4' THEN '45' 
		 WHEN '890d0180-5c01-11ed-b331-2b7bf508673' THEN '45' 
	END 
	status = CASE id 
		 WHEN '6e49474c-8103-11ed-8410-ebb084c2a71f' THEN 'IN_PROGRESS' 
		 WHEN '742b0f74-8103-11ed-8410-ebb084c2a71f' THEN 'IN_PROGRESS' 
		 WHEN 'a67d5688-5ee1-11ed-9f7e-5fce40501278' THEN 'IN_PROGRESS' 
		 WHEN 'd2a19802-4fbc-11ed-9c57-a181204819a4' THEN 'IN_PROGRESS' 
		 WHEN '890d

In [1]:
import boto3

In [2]:
boto3.setup_default_session(
    profile_name="kaushik",
    region_name="us-east-1"
    )


In [9]:
s3 = boto3.resource('s3')

# Get bucket object
my_bucket = s3.Bucket('ml-stack.prod')
# Iterate over objects in bucket
insertion_s3_object_key = []
updation_s3_object_key = []
for obj in my_bucket.objects.filter(Prefix="brand_normalization"):
    
    if 'insert' in obj.key:
        insertion_s3_object_key.append(obj.key)
    elif 'update' in obj.key:
        updation_s3_object_key.append(obj.key)


In [10]:
print(insertion_s3_object_key)
print(updation_s3_object_key)

['brand_normalization/leftover_brands/insert/part-00000-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00001-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00002-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00003-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00004-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00005-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00006-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00007-2ea9511c-dcaf-4e76-b158-975a662fd038-c000.snappy.parquet', 'brand_normalization/leftover_brands/insert/part-00008-2ea9511c-dcaf-4e76-b158-975a662f

In [7]:
path = '/home/preacher/Bungee/bnt/upodate_brands.parquet'

In [8]:
df = pd.read_parquet(path)

In [9]:
df_sample = df.head(10)

In [10]:
df_sample

,cluster_id,crawled_name,source,status,updated_date,product_url_1,image_url_1,product_url_2,query_type
0,-1,"""0""""isAjaxComplete_B001IOHA8M""",UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
1,-1,"""TBA""",UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
2,-1,"""YelloWay""",UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
3,-1,'jade-collector.com',UC,IN_PROGRESS,2023-01-09 13:19:13.694,https://amazon.com/dp/b09sp2877f,https://m.media-amazon.com/images/i/510lu8zfhj...,https://amazon.com/dp/b09sv9dpwp,I
4,-1,($13 Value) AXE 4-pc Apollo Holiday Gift Set (...,UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
5,-1,(2 Pack) Salon Perfect Nail Lacquer - Under Th...,UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
6,-1,(6 Packs) Plum Organics,UC,IN_PROGRESS,2023-01-09 13:19:13.694,None,None,None,I
7,-1,(Absport),UC,IN_PROGRESS,2023-01-09 13:19:13.694,https://amazon.com/dp/b07pjh4fxf,https://m.media-amazon.com/images/i/61g-7o6lat...,https://amazon.com/dp/b07pjh4fxf,I
8,-1,(FINE JAPAN),UC,IN_PROGRESS,2023-01-09 13:19:13.694,https://amazon.com/dp/b01ibkrwdi,https://m.media-amazon.com/images/i/71hoqqnieu...,https://amazon.com/dp/b01ibkrwdi,I
9,-1,(MAX),UC,IN_PROGRESS,2023-01-09 13:19:13.694,https://amazon.com/dp/b000fhpuas,https://m.media-amazon.com/images/i/61xlc9ktw3...,https://amazon.com/dp/b000fhpuas,I


In [11]:
df_sample.to_parquet('/home/preacher/Bungee/bnt/sample_update_brands.parquet')